## Scrap of Hyeres's climate from january 2020 to december 2021.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import itertools as it
import random
from datetime import datetime
import time
import bs4
import numpy as np
from scipy.ndimage.interpolation import shift
#!pip install fake-useragent
#from fake_useragent import UserAgent

### Format of the website

In [2]:
regions = ["provence-alpes-c-te-d-azur","Occitanie","Nouvelle-Aquitaine","auvergne-rhone-alpes","Ile-de-France"]
ville = ["Marseille","Toulouse","Bordeaux","Lyon","Paris"]

In [3]:
#url = "https://www.historique-meteo.net/france/region/ville/"
url = "https://www.historique-meteo.net//france/aquitaine/bordeaux/"

### range date of data we want to get

In [4]:
date_debut = '2020/01/01'
date_fin ='2021/12/31'

##### Getting all the url from date_debut to date_fin

In [5]:
def get_pages(url,deb,fin):
    
    # getting the date range
    def get_rangeDates(deb,fin):
        dateTime = pd.date_range(start=deb, end=fin)
        date = []
        for i in dateTime:
            i = i.date() # getting the date without the time
            i = str(i) # converting to str to use the replace function
            date.append(i)
        return(date)
    pages =[]
    dates = get_rangeDates(deb,fin) 
    for i in dates:
        i = i.replace('-','/') # To have the same date format of the website
        j = url + str(i)
        pages.append(j)
    return(pages)

In [6]:
pages = get_pages(url,date_debut,date_fin)
print(len(pages))
print(pages[-1])

731
https://www.historique-meteo.net//france/aquitaine/bordeaux/2021/12/31


### Getting the data

In [7]:
# fUNCTIONC TO GET ALL DATAS 
def get_all_data(pages):
    def get_data(url): # creating the get_data function for one url
        Reponse = requests.get(url)
        soup = BeautifulSoup(Reponse.text, "html.parser")
        table = soup.find("table",{"class":"table"})
        df = []
        for valeur in table.find_all('tbody'):
            rows = table.find_all('tr')
            for row in rows:
                donnee = row.find('td', class_ ="text-center bg-primary")
                # the last row of our table (avis) don't have values (None type)
                # so we add a string in our list (fefekeff) to replace them.
                if donnee != None: 
                    donnee= donnee.text.strip()
                else:
                    donnee = 'fefekeff'
                df.append(donnee)
        return(df)  
    df =  []
    for i in pages: # calling the get_data function for all urls
        time.sleep(random.randrange(1,5))
        dataset = get_data(i)# calling the get data function
        df.append(dataset)   
    return df

In [8]:
data = get_all_data(pages)
#print(len(data))
#data[-1]

In [9]:
# shift(data[-1], -1)

### Creating the dataframe to store extracted data

In [10]:
column = ['Dates','Tempmax_°C','Tempmin_°C','Vitesse_vent_km/h','Temp_vent_°C','Precipitation_mm',
           'Humidite_%','Visibilite_km','Couverture_nuageuse_%','Indice_chaleur','Point_rosee_°C',
           'Pression_hpa','lever_soleil','coucher_soleil','duree_jour','avis']
indexes   = pd.date_range(start=date_debut, end=date_fin)

In [11]:
def store_data(data,col): 
    dataset = pd.DataFrame(data,columns=col) # we will use the indexes at the end 
    return(dataset)

In [12]:
dataset = store_data(data,column)
print(dataset.shape)
dataset.head()

(731, 16)


,Dates,Tempmax_°C,Tempmin_°C,Vitesse_vent_km/h,Temp_vent_°C,Precipitation_mm,Humidite_%,Visibilite_km,Couverture_nuageuse_%,Indice_chaleur,Point_rosee_°C,Pression_hpa,lever_soleil,coucher_soleil,duree_jour,avis
0,01/01/2020,11°,10°,10km/h,9°,2mm,85%,9km,82%,11,9°C,1032hPa,08:40:00,17:32:00,8:52:0,fefekeff
1,02/01/2020,11°,7°,11km/h,6°,1mm,94%,9km,57%,11,8°C,1031hPa,08:40:00,17:33:00,8:53:0,fefekeff
2,03/01/2020,12°,10°,12km/h,9°,2mm,95%,6.625km,81%,12,10°C,1034hPa,08:40:00,17:34:00,8:54:0,fefekeff
3,04/01/2020,10°,7°,9km/h,5°,0mm,95%,6.625km,26%,10,9°C,1037hPa,08:40:00,17:35:00,8:55:0,fefekeff
4,05/01/2020,9°,4°,10km/h,1°,83%,10km,3%,9,4°C,1037hPa,08:40:00,17:36:00,8:56:0,fefekeff,None


### Correction of our dataset

##### if there is no column precipitation( precipitation =0 mm), our table is shift 
filter shifted lines in dataframe, shift them in the other sens to correct them, deleate the shifted lines in the whole dataset and replace them by the corrected one.
Then we sort values by date and now we can set indexes.

In [13]:
def replace_data(dataset,col):
    na_ligne = pd.DataFrame(dataset[dataset.isnull().any(axis = 1)]) # getting  the shiffeted line 
    def decal_data(na_ligne):
    # we'll trunc the dataset because the shift function put the none value at the first position of the list and here we want none at the Precipitations column.
        correct_val = []
        for i in range(len(na_ligne)):
            df = na_ligne.iloc[i,:]
            df_coupe_debut = df[:'Temp_vent_°C']
            df_coupe_fin = df.loc['Precipitation_mm':]
            df_coupe_fin = df_coupe_fin.shift(periods=1,fill_value = 0)
            df = list(it.chain(df_coupe_debut, df_coupe_fin))
            correct_val.append(df)
        return(correct_val)
    
    correct_val = decal_data(na_ligne)
    correct_val = pd.DataFrame(correct_val,columns = col)
    cleaned_dataset = dataset.dropna()
    cleaned_dataset = cleaned_dataset.append(correct_val)
    cleaned_dataset.reset_index()
    cleaned_dataset.sort_values(by=['Dates'], inplace = True)
    return(cleaned_dataset)

In [14]:
cleaned_dataset = replace_data(dataset,column)
cleaned_dataset = cleaned_dataset.set_index(indexes)
cleaned_dataset.head()

,Dates,Tempmax_°C,Tempmin_°C,Vitesse_vent_km/h,Temp_vent_°C,Precipitation_mm,Humidite_%,Visibilite_km,Couverture_nuageuse_%,Indice_chaleur,Point_rosee_°C,Pression_hpa,lever_soleil,coucher_soleil,duree_jour,avis
2020-01-01,01/01/2020,11°,10°,10km/h,9°,2mm,85%,9km,82%,11,9°C,1032hPa,08:40:00,17:32:00,8:52:0,fefekeff
2020-01-02,01/01/2021,7°,4°,8km/h,3°,0mm,87%,10km,27%,7,2°C,1012hPa,08:40:00,17:32:00,8:52:0,fefekeff
2020-01-03,01/02/2020,14°,12°,17km/h,11°,3mm,94%,8.125km,82%,14,13°C,1023hPa,08:22:00,18:10:00,9:48:0,fefekeff
2020-01-04,01/02/2021,14°,11°,32km/h,8°,3mm,84%,9.25km,94%,14,11°C,1011hPa,08:21:00,18:11:00,9:50:0,fefekeff
2020-01-05,01/03/2020,12°,8°,32km/h,6°,8mm,82%,9km,74%,12,6°C,1010hPa,07:40:00,18:50:00,11:10:0,fefekeff


In [15]:
cleaned_dataset.to_csv("data/MeteotoulBrut.csv")

Now we'll remove caracters in our dats('° % km/h °C hPa')

In [16]:
def remove_carcter(dataset):
    for i in dataset.columns:
        dataset[i] = dataset[i].str.strip('° % km/h °C hPa')
    return dataset



In [17]:
final_daset = remove_carcter(cleaned_dataset)
final_daset

,Dates,Tempmax_°C,Tempmin_°C,Vitesse_vent_km/h,Temp_vent_°C,Precipitation_mm,Humidite_%,Visibilite_km,Couverture_nuageuse_%,Indice_chaleur,Point_rosee_°C,Pression_hpa,lever_soleil,coucher_soleil,duree_jour,avis
2020-01-01,01/01/2020,11,10,10,9,2,85,9,82,11,9,1032,08:40:00,17:32:00,8:52:0,fefekeff
2020-01-02,01/01/2021,7,4,8,3,0,87,10,27,7,2,1012,08:40:00,17:32:00,8:52:0,fefekeff
2020-01-03,01/02/2020,14,12,17,11,3,94,8.125,82,14,13,1023,08:22:00,18:10:00,9:48:0,fefekeff
2020-01-04,01/02/2021,14,11,32,8,3,84,9.25,94,14,11,1011,08:21:00,18:11:00,9:50:0,fefekeff
2020-01-05,01/03/2020,12,8,32,6,8,82,9,74,12,6,1010,07:40:00,18:50:00,11:10:0,fefekeff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,31/08/2021,33,15,12,14,NaN,74,10,13,32,12,1018,06:23:00,19:41:00,13:18:0,fefekeff
2021-12-28,31/10/2020,21,12,9,11,0,85,10,27,21,11,1022,07:40:00,17:52:00,10:12:0,fefekeff
2021-12-29,31/10/2021,23,14,22,13,2,90,8.875,60,25,15,1011,07:40:00,17:52:00,10:12:0,fefekeff
2021-12-30,31/12/2020,10,9,21,6,9,82,9.125,77,10,7,1011,08:40:00,17:31:00,8:51:0,fefekeff


In [18]:
# selecting columns which interest us and remove NaNs

final_daset = final_daset [['Dates', 'Tempmax_°C', 'Tempmin_°C', 'Vitesse_vent_km/h','Precipitation_mm', 'Humidite_%','Pression_hpa']]
final_daset = final_daset.fillna(0)
print(final_daset.shape)
final_daset.head()

(731, 7)


,Dates,Tempmax_°C,Tempmin_°C,Vitesse_vent_km/h,Precipitation_mm,Humidite_%,Pression_hpa
2020-01-01,01/01/2020,11,10,10,2,85,1032
2020-01-02,01/01/2021,7,4,8,0,87,1012
2020-01-03,01/02/2020,14,12,17,3,94,1023
2020-01-04,01/02/2021,14,11,32,3,84,1011
2020-01-05,01/03/2020,12,8,32,8,82,1010


In [19]:
final_daset.to_csv("data/MeteoToulouse.csv")